# Monte Carlo Uncertainty Quantification

This notebook demonstrates Monte Carlo ensemble simulations for uncertainty quantification.

## Learning Objectives

- Define parameter uncertainties
- Run ensemble simulations
- Statistical analysis of results
- Landing dispersion analysis
- Export for sensitivity analysis

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from src.config_loader import ConfigLoader
from src.monte_carlo_runner import MonteCarloRunner

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ All imports successful")

## Part 1: Setup and Configure Uncertainties

In [ ]:
config_path = Path('../configs/complete_example.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")

In [ ]:
config_path = Path('../configs/monte_carlo/01_basic_mc.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")

## Part 2: Run Ensemble Simulation

In [ ]:
print("Running Monte Carlo simulation...")
results = mc_runner.run(parallel=True, max_workers=4)

print(f"\nComplete: {len(results)}/{mc_runner.num_simulations} successful")
mc_runner.print_statistics_summary()

## Part 3: Analyze Results

In [ ]:
stats = mc_runner.get_statistics()

# Plot apogee distribution
apogees = [r['apogee_m'] for r in results]

plt.figure(figsize=(10, 6))
plt.hist(apogees, bins=30, alpha=0.7, edgecolor='black')
plt.axvline(stats['apogee_m']['mean'], color='red', linestyle='--', 
            label=f"Mean: {stats['apogee_m']['mean']:.1f} m")
plt.axvline(stats['apogee_m']['p05'], color='orange', linestyle=':', 
            label=f"5th %ile: {stats['apogee_m']['p05']:.1f} m")
plt.axvline(stats['apogee_m']['p95'], color='orange', linestyle=':', 
            label=f"95th %ile: {stats['apogee_m']['p95']:.1f} m")
plt.xlabel('Apogee (m)')
plt.ylabel('Frequency')
plt.title('Apogee Distribution - Monte Carlo Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Landing dispersion
x_impacts = [r['x_impact_m'] for r in results]
y_impacts = [r['y_impact_m'] for r in results]

plt.figure(figsize=(8, 8))
plt.scatter(x_impacts, y_impacts, alpha=0.5)
plt.axhline(0, color='k', linestyle='--', alpha=0.3)
plt.axvline(0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('X Impact (m)')
plt.ylabel('Y Impact (m)')
plt.title('Landing Dispersion Ellipse')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

## Part 4: Export for Sensitivity Analysis

In [ ]:
# Export in RocketPy format for sensitivity analysis
output_dir = Path('../outputs/monte_carlo')

input_path, output_path = mc_runner.save_rocketpy_format(
    output_dir=str(output_dir),
    filename_prefix="mc_results"
)

print(f"Data exported:")
print(f"  {input_path}")
print(f"  {output_path}")
print("\nReady for sensitivity analysis! See notebook 03.")